## TensorFlow

## Практическое задание

  1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора)  на датасет from sklearn.datasets import load_boston. 
  2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

  3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

  1-2. (*)  Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews .Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?
    
    
</ol>



## Набор данных для последнего ДЗ

- COVID 19 диагностика https://www.kaggle.com/andrewmvd/covid19-ct-scans

- Распознавание изображений цифр в представлении языка жестов https://www.kaggle.com/ardamavi/sign-language-digits-dataset

In [1]:
import numpy as np
import pandas as pd
import itertools

import tensorflow as tf

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam

print(tf.__version__)

2.3.0


In [3]:
from tensorflow.keras.datasets import boston_housing

In [4]:
boston = load_boston()

X = boston["data"]
y = boston["target"]
feature_names = np.append(boston["feature_names"], ['target']) 

data = pd.DataFrame(data=np.c_[X, y], columns=feature_names)
data.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(13,)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='Adam',
                     loss='mse',
                     metrics=['mse'])   

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(
    X_train,
    y_train,
    epochs=5,
    batch_size=20,
    verbose=0    
)

In [8]:
model.evaluate(X_train, y_train)

13/13 [==============================] - 0s 462us/step - loss: 68.1052 - mse: 68.1052


[68.10523223876953, 68.10523223876953]

In [9]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 500us/step - loss: 90.1354 - mse: 90.1354


[90.13536071777344, 90.13536071777344]

In [10]:
predictions = model.predict(X_test)

In [11]:
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2

In [12]:
r2(y_test, predictions)

0.10303573636284791

In [15]:
def create_model(layers, neurons, activator):
    model = Sequential()
    model.add(Dense(neurons, activation=activator, input_shape=(13,)))
    for k in range(layers - 2):
        model.add(Dense(neurons * (10-k) * 0.06, activation=activator, input_shape=(13,)))
    model.add(Dense(1, activation=activator))
    return model

In [16]:
results = []
for opt in tqdm(['Adam', 'RMSprop', 'nadam']):
    for act in tqdm(['relu', 'selu', 'linear']):
        for n in tqdm([8, 16, 32, 64, 128,]):
                for loss in tqdm(['mae', 'mse']):
                    for l in tqdm(range(1, 5)):

                        model = create_model(l, n, act)                   
                        i = 0

                        result = {}

                        model.compile(optimizer=opt,
                                        loss=loss,
                                        metrics=[loss])

                        model.fit(
                            X_train, 
                            y_train, 
                            epochs=50, 
                            batch_size=20, 
                            verbose = False)
                        y_pred = model.predict(X_test)

                        result['n'] = n
                        result['opt'] = opt
                        result['loss'] = loss
                        result['layers'] = l
                        result['activation'] = act
                        i += 1
                        result['r2'] = round(r2(y_test, y_pred.copy().flatten()),5)
                        result['rmse'] = round(np.sqrt(mse(y_test, y_pred.copy().flatten())),5)
                        result['mse'] = round(mse(y_test, y_pred.copy().flatten()),5)
                        result['mae'] = round(mae(y_test, y_pred.copy().flatten()),5)


                        results.append(result)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
result = pd.DataFrame(results)
result.head(10)

,n,opt,loss,layers,activation,r2,rmse,mse,mae
0,8,Adam,mae,1,relu,0.32616,8.22882,67.71348,5.04361
1,8,Adam,mae,2,relu,0.42595,7.59515,57.68632,4.60482
2,8,Adam,mae,3,relu,-5.49928,25.55598,653.10804,23.50784
3,8,Adam,mae,4,relu,0.22665,8.81554,77.71376,5.56263
4,8,Adam,mse,1,relu,-5.49928,25.55598,653.10804,23.50784
5,8,Adam,mse,2,relu,-5.49928,25.55598,653.10804,23.50784
6,8,Adam,mse,3,relu,-5.49928,25.55598,653.10804,23.50784
7,8,Adam,mse,4,relu,-5.49928,25.55598,653.10804,23.50784
8,16,Adam,mae,1,relu,-5.49928,25.55598,653.10804,23.50784
9,16,Adam,mae,2,relu,0.40122,7.75702,60.17129,5.29313


__Посмотрим на лучшие результаты__

In [18]:
result = pd.DataFrame(results).sort_values('r2', ascending=False)
result.head(10)

,n,opt,loss,layers,activation,r2,rmse,mse,mae
308,64,nadam,mse,1,selu,0.69529,5.53358,30.62046,3.98685
60,32,Adam,mse,1,selu,0.68575,5.61954,31.57919,4.09280
303,32,nadam,mse,4,selu,0.65675,5.87305,34.49274,3.91194
116,128,Adam,mse,1,linear,0.65438,5.89334,34.73146,4.31793
76,128,Adam,mse,1,selu,0.65212,5.91252,34.95789,4.05131
65,64,Adam,mae,2,selu,0.64633,5.96158,35.54049,4.08304
30,64,Adam,mse,3,relu,0.64611,5.96338,35.56191,4.15680
69,64,Adam,mse,2,selu,0.63562,6.05112,36.61603,4.14347
28,64,Adam,mse,1,relu,0.63133,6.08664,37.04722,4.30768
277,128,nadam,mse,2,relu,0.62948,6.10193,37.23359,4.61137


__Посмотрим кто оказался в конце__

In [19]:
result.tail(10)

,n,opt,loss,layers,activation,r2,rmse,mse,mae
66,64,Adam,mae,3,selu,-6.35259,27.18193,738.8572,25.26594
64,64,Adam,mae,1,selu,-6.35259,27.18193,738.8572,25.26594
57,32,Adam,mae,2,selu,-6.35259,27.18193,738.8572,25.26594
186,64,RMSprop,mae,3,selu,-6.35259,27.18193,738.8572,25.26594
51,16,Adam,mae,4,selu,-6.35259,27.18193,738.8572,25.26594
164,8,RMSprop,mse,1,selu,-6.35259,27.18193,738.8572,25.26594
192,128,RMSprop,mae,1,selu,-6.35259,27.18193,738.8572,25.26594
172,16,RMSprop,mse,1,selu,-6.35259,27.18193,738.8572,25.26594
193,128,RMSprop,mae,2,selu,-6.35259,27.18193,738.8572,25.26594
180,32,RMSprop,mse,1,selu,-6.35259,27.18193,738.8572,25.26594


По сравнению с первоначальным результатом, сеть значительно улучшилась и показала неплохие результаты. 

Мы попробовали разные оптимизаторы, функции потерь, слои, активаторы и увидели результаты, на которые они повлияли. 

__Самый лучший результат у нас получился при количестве нейронов равном 64, 1 слой, оптимизатор nadam и активатор selu.__

__Самый худший результат был получени при количестве нейронов равном 32, 1 слой, оптимизатор RMSprop и активатор selu.__

Также мы использовали одно количество эпох, что тоже повлияло на результат, с перебором эпох, думаю результат был бы немного другим. 